In [1]:
import darkwing as dw

def foo(rel, res=6):
    return (rel
    | 'select pickup_latitude as lat, pickup_longitude as lng, tip_amount'       
    | f'select h3_latlng_to_cell(lat, lng, {res}) as hexid, tip_amount as tip'
    | 'select hexid, avg(tip) as tip group by 1'
    | 'select h3_h3_to_string(hexid) as hexid, tip'
    | 'where tip > 0'
    )

In [2]:
rel1 = dw.load('data/yellow_tripdata_2010-01.parquet') # lazy load!
rel2 = dw.load('data/yellow_tripdata_2010-02.parquet') # lazy load!

In [3]:
# TODO: why isn't this lazy?
# rel3 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-01.parquet' >> dw.load

In [4]:
rel1 | foo

┌─────────────────┬────────────────────┐
│      hexid      │        tip         │
│     varchar     │       double       │
├─────────────────┼────────────────────┤
│ 862a100d7ffffff │  0.604239909636192 │
│ 862a10727ffffff │ 0.6621288459090504 │
│ 862a1008fffffff │ 0.5471456491468885 │
│ 86754e64fffffff │ 0.6730361779149294 │
│ 862a100e7ffffff │ 1.9770422191887693 │
│ 862a10387ffffff │ 1.5341203703703703 │
│ 862b1b2b7ffffff │                2.0 │
│ 862a10c5fffffff │  6.260140845070423 │
│ 862a30297ffffff │ 1.5257142857142856 │
│ 862a15007ffffff │ 0.6904761904761905 │
│        ·        │                 ·  │
│        ·        │                 ·  │
│        ·        │                 ·  │
│ 862a13067ffffff │               15.0 │
│ 862a10aefffffff │                3.5 │
│ 862aa08c7ffffff │                1.0 │
│ 862aa556fffffff │                1.0 │
│ 862a022b7ffffff │                3.0 │
│ 862aac60fffffff │                2.0 │
│ 8604602f7ffffff │ 0.6666666666666666 │
│ 862a12137fffff

In [10]:
out = dw.Database(
    rel1 = rel1 >> foo,
    rel2 = rel2 >> foo,
) | """
select
    hexid,
    rel1.tip
        as tip1,
    rel2.tip
        as tip2,
from rel1
full outer join rel2
using (hexid)
"""

(out
| 'where tip1 is not null and tip2 is not null'
| 'select *, tip1-tip2 as diff order by diff'
) >> 'as bam' # precedence rules!

Database:
    bam: 371 x ['hexid', 'tip1', 'tip2', 'diff']